In [1]:
import sys
import requests
import json

In [2]:
from bs4 import BeautifulSoup as bs
from contextlib import closing
import urllib
import re

# BeautifulSoup
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

<html>
    <head>
        <title>The Dormouse's story</title>
    </head>
    <body>
        <p class="title" name="dromouse">
            <b>
                The Dormouse's story
            </b>
        </p>
        <p class="story">
        Once upon a time there were three little sisters; and their names were
            <a href="http://example.com/elsie" class="sister" id="link1">
            <!-- Elsie -->
            </a>,
            <a href="http://example.com/lacie" class="sister" id="link2">
            Lacie
            </a> 
            and
            <a href="http://example.com/tillie" class="sister" id="link3">
            Tillie
            </a>;
            and they lived at the bottom of a well.
        </p>
        <p class="story">...</p>
    </body>
</html>

[Internal Node] head
[Internal Node] title
[Leaf Node] The Dormouse's title
[Internal Node] body
[Internal Node] p
[Internal Node] b
[Leaf Node] The Dormouse's story
[Internal Node] p
[Leaf Node] Once upon a time there were three little sisters; and their names were

[Internal Node] a
[Leaf Node]  Elsie 
[Leaf Node] ,

[Internal Node] a
[Leaf Node] Lacie
[Leaf Node]  and

[Internal Node] a
[Leaf Node] Tillie
[Leaf Node] ;
and they lived at the bottom of a well.
[Internal Node] p
[Leaf Node] ...

In [42]:
html = """
<html><head><title>The Dormouse's title</title></head>
<body>
<p class="title" name="dromouse"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1"><!-- Elsie --></a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
</body>
</html>
"""

In [4]:
?bs

In [43]:
soup = bs(html,'html.parser')

In [44]:
print(soup.prettify())

<html>
 <head>
  <title>
   The Dormouse's title
  </title>
 </head>
 <body>
  <p class="title" name="dromouse">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    <!-- Elsie -->
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>



In [8]:
soup.text

"\nThe Dormouse's title\n\nThe Dormouse's story\nOnce upon a time there were three little sisters; and their names were\n,\nLacie and\nTillie;\nand they lived at the bottom of a well.\n...\n\n\n"

In [37]:
?soup.find_all

In [50]:
all_a = soup.find_all("a")
print(all_a)
print(type(all_a))


[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>, <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>, <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]
<class 'bs4.element.ResultSet'>


In [41]:
all_a[2]["href"]

'/what-unions-do'

In [51]:
type(soup.p)

bs4.element.Tag

In [53]:
soup.a['class']

['sister']

In [17]:
# Depth First Search
def recursiveChildren(x):
    if "childGenerator" in dir(x):
        for child in x.childGenerator():
            name = getattr(child, "name", None)
            if name is not None:
                print("[Internal Node]",child.name)
            recursiveChildren(child)
    else:
        if not x.isspace(): #Just to avoid printing "\n" parsed from document.
            print("[Leaf Node]",x)

In [18]:
soup.text

"\nThe Dormouse's title\n\nThe Dormouse's story\nOnce upon a time there were three little sisters; and their names were\n,\nLacie and\nTillie;\nand they lived at the bottom of a well.\n...\n\n\n"

In [19]:
for child in soup.childGenerator():
    recursiveChildren(child)

[Internal Node] head
[Internal Node] title
[Leaf Node] The Dormouse's title
[Internal Node] body
[Internal Node] p
[Internal Node] b
[Leaf Node] The Dormouse's story
[Internal Node] p
[Leaf Node] Once upon a time there were three little sisters; and their names were

[Internal Node] a
[Leaf Node]  Elsie 
[Leaf Node] ,

[Internal Node] a
[Leaf Node] Lacie
[Leaf Node]  and

[Internal Node] a
[Leaf Node] Tillie
[Leaf Node] ;
and they lived at the bottom of a well.
[Internal Node] p
[Leaf Node] ...


In [20]:
soup.title

<title>The Dormouse's title</title>

In [21]:
soup.head

<head><title>The Dormouse's title</title></head>

In [22]:
soup.a

<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>

In [23]:
soup.p

<p class="title" name="dromouse"><b>The Dormouse's story</b></p>

In [25]:
#NavigableString
print(soup.p.string)
print(type(soup.p.text))
print(type(soup.p.string))

The Dormouse's story
<class 'str'>
<class 'bs4.element.NavigableString'>


In [27]:
#（3）BeautifulSoup
soup.name

'[document]'

In [24]:
#Comment
#print(soup.a)
print(soup.a.string)
#print(type(soup.a.string))

 Elsie 


In [17]:
soup.find_all("a")

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

### HTML and the DOM
The general idea behind web scraping is to retrieve data that exists on a website, and convert it into a format that is usable for analysis. Webpages are rendered by the brower from HTML and CSS code, but much of the information included in the HTML underlying any website is not interesting to us.
We begin by reading in the source code for a given web page and creating a Beautiful Soup object with the BeautifulSoup function.

In [35]:
from bs4 import BeautifulSoup
import urllib.request 
import requests

r = requests.get('http://www.aflcio.org/Legislation-and-Politics/Legislative-Alerts')

soup = BeautifulSoup(r.text)
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [36]:
soup

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
<head>
<meta charset="utf-8"/><script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(3),u=e(4),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;

In [32]:
soup.name

'[document]'

![](HTMLDOMTree.png)
![](treeStructure.png)

In [33]:
print(soup.prettify()[28700:30500])


">
                   Legislative Alert
                  </h5>
                  <h2 class="content-title">
                   <span>
                    Letter Opposing Nomination of Daniel Collins to Appeals Court
                   </span>
                  </h2>
                  <time datetime="2019-05-21T12:30:01-0400">
                   May 21, 2019
                  </time>
                 </div>
                </a>
                <div>
                </div>
               </div>
              </div>
              <div class="block block-content col-12 col-lg-4">
               <div class="content-details ">
                <a class="b-inner" href="/aboutadvocacylegislative-alerts/letter-senators-opposing-nomination-kenneth-lee-us-court-appeals">
                 <div class="b-text">
                  <h5 class="content-type">
                   Legislative Alert
                  </h5>
                  <h2 class="content-title">
                   <span>
               

In [34]:
letters = soup.find_all("div", class_="content-details")

In [35]:
letters

[<div class="content-details ">
 <a class="b-inner" href="/aboutadvocacylegislative-alerts/letter-representatives-support-dream-and-american-promise-act-2019">
 <div class="b-text">
 <h5 class="content-type">Legislative Alert</h5>
 <h2 class="content-title"><span>Letter to Representatives in Support of the Dream and American Promise Act of 2019</span>
 </h2>
 <time datetime="2019-06-03T13:51:36-0400">June 3, 2019</time>
 </div>
 </a>
 <div></div>
 </div>, <div class="content-details ">
 <a class="b-inner" href="/aboutadvocacylegislative-alerts/letter-senators-opposing-nomination-rossie-alston-us-court-appeals">
 <div class="b-text">
 <h5 class="content-type">Legislative Alert</h5>
 <h2 class="content-title"><span>Letter to Senators Opposing the Nomination of Rossie Alston to the U.S. Court of Appeals for the Eastern District of Virginia</span>
 </h2>
 <time datetime="2019-06-03T13:35:45-0400">June 3, 2019</time>
 </div>
 </a>
 <div></div>
 </div>, <div class="content-details ">
 <a cla

In [36]:
print(type(letters))

<class 'bs4.element.ResultSet'>


In [37]:
letters[0]

<div class="content-details ">
<a class="b-inner" href="/aboutadvocacylegislative-alerts/letter-representatives-support-dream-and-american-promise-act-2019">
<div class="b-text">
<h5 class="content-type">Legislative Alert</h5>
<h2 class="content-title"><span>Letter to Representatives in Support of the Dream and American Promise Act of 2019</span>
</h2>
<time datetime="2019-06-03T13:51:36-0400">June 3, 2019</time>
</div>
</a>
<div></div>
</div>

In [60]:
lobbying = {}
for element in letters:
    lobbying[element.a.get_text()] = {}

In [56]:
print(letters[0].a.get_text())



Legislative Alert
Letter to Representatives in Support of the Dream and American Promise Act of 2019

June 3, 2019




In [39]:
letters[0].a["href"]

'/aboutadvocacylegislative-alerts/letter-representatives-support-dream-and-american-promise-act-2019'

In [61]:
prefix = "www.aflcio.org"

In [62]:
for element in letters:
    lobbying[element.a.get_text()]["link"] = prefix + element.a["href"]

In [59]:
letters[0].find("time")

<time datetime="2019-06-03T13:51:36-0400">June 3, 2019</time>

In [63]:
for element in letters:
    date = element.find("time").get_text()
    lobbying[element.a.get_text()]["date"] = date

In [64]:
for item in lobbying.keys():
    print(item + ": " + "\n\t" + "link: " + lobbying[item]["link"] + "\n\t" + "date: " + lobbying[item]["date"] + "\n\n" )



Legislative Alert
Letter to Representatives in Support of the Dream and American Promise Act of 2019

June 3, 2019

: 
	link: www.aflcio.org/aboutadvocacylegislative-alerts/letter-representatives-support-dream-and-american-promise-act-2019
	date: June 3, 2019




Legislative Alert
Letter to Senators Opposing the Nomination of Rossie Alston to the U.S. Court of Appeals for the Eastern District of Virginia

June 3, 2019

: 
	link: www.aflcio.org/aboutadvocacylegislative-alerts/letter-senators-opposing-nomination-rossie-alston-us-court-appeals
	date: June 3, 2019




Legislative Alert
Letter to Senators Opposing the Nomination of Matthew Kacsmaryk to the U.S. District Court for the Northern District of Texas

May 31, 2019

: 
	link: www.aflcio.org/aboutadvocacylegislative-alerts/letter-senators-opposing-nomination-matthew-kacsmaryk-us-district
	date: May 31, 2019




Legislative Alert
Letter Opposing Howard Nielson's Appointment to District Court

May 21, 2019

: 
	link: www.aflcio.org/

In [74]:
import os, csv

with open("lobbying1.csv", "w") as toWrite:
    writer = csv.writer(toWrite)#, delimiter=",")
    writer.writerow(["name", "link", "date"])
    for a in lobbying.keys():
        writer.writerow([str(a.encode("utf-8")), lobbying[a]["link"], lobbying[a]["date"]])

In [71]:
?csv.writer

In [ ]:
import json

with open("lobbying.json", "w") as writeJSON:
    json.dump(lobbying, writeJSON)